<b>Local Magnitude Estimator</b><br>
This Notebook is designed to demonstrate how to download a set of waveforms from stations near a catalog earthquake and use them to estimate the local magnitude.


In [13]:
import os
from math import log10
from obspy import UTCDateTime, read, read_inventory
from obspy.geodetics import gps2dist_azimuth
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader
from obspy.clients.fdsn import Client
client = Client("IRIS")

The following code will set the variables for the earthquake catalog search.  If a catalog event is not yet available, the values needed for future code are the origin time and the latitude and longitude.<br><br>
In the predefined example, the values are set for a small earthquake in the Eastern Tennessee Seismic Zone.  The USGS page for this event can be found here:<br> <a href=https://earthquake.usgs.gov/earthquakes/eventpage/se60300191>https://earthquake.usgs.gov/earthquakes/eventpage/se60300191</a>

In [26]:
catstartt = UTCDateTime("2020-01-01")
catendt = UTCDateTime("2020-08-15")
lon = -84
lat = 36.43
rad = 1
cat = client.get_events(starttime=catstartt, endtime=catendt, latitude=lat, longitude=lon, maxradius=rad, minmagnitude=3)

Printing the catalog will show the events which have been returned by the search.  Note what the catalog magnitude value is and what type it is.  The origin time is stored in a variable.

In [24]:
print(cat)

1 Event(s) in Catalog:
2020-01-20T19:12:11.260000Z | +36.431,  -84.027 | 3.8 Mwr


Next we set the variables for the waveform search.

In [28]:
origint = UTCDateTime(cat[0].origins[0].time.datetime)
wfstartt = origint - 10
wfendt = origint + 50
stamaxrad = 1
domain = CircularDomain(latitude=lat, longitude=lon, minradius=0, maxradius=stamaxrad)
restrictions = Restrictions(starttime=wfstartt, endtime=wfendt, channel_priorities=["HH[NE12]", "BH[NE12]"], location_priorities=["", "00", "10"])

The following code performs the waveform search.  It looks for stations from many different FDSN clients, so it can take several minutes to complete.  It will produce a lot of information in pink that describes the different steps through the search process.

In [6]:
mdl = MassDownloader()
mdl.download(domain,restrictions, mseed_storage="waveforms", stationxml_storage="stations")

[2020-08-15 19:04:18,844] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for BGR, EMSC, ETH, GEONET, GFZ, ICGC, INGV, IPGP, ISC, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, SCEDC, TEXNET, USGS, USP, ORFEUS, IRIS.
[2020-08-15 19:04:20,722] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'ISC' as it does not have 'dataselect' and/or 'station' services.
[2020-08-15 19:04:21,119] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'USGS' as it does not have 'dataselect' and/or 'station' services.
[2020-08-15 19:04:21,165] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'EMSC' as it does not have 'dataselect' and/or 'station' services.
[2020-08-15 19:04:21,480] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 19 client(s): BGR, ETH, GEONET, GFZ, ICGC, INGV, IPGP, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, SCEDC, TEXNET, USP, ORFEUS, IRIS.
[2020-08-15 19:04:21,500] - obspy.clients.fdsn.mass_downloader - IN

[2020-08-15 19:05:31,187] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2020-08-15 19:05:31,191] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - Requesting unreliable availability.
[2020-08-15 19:05:32,030] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - No data available for request.
[2020-08-15 19:05:32,047] - obspy.clients.fdsn.mass_downloader - INFO: Client 'USP' - No data available.
[2020-08-15 19:05:32,052] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2020-08-15 19:05:32,055] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ORFEUS' - Requesting unreliable availability.
[2020-08-15 19:05:33,094] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ORFEUS' - No data available for request.
[2020-08-15 19:05:33,096] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ORFEUS' - No data available.
[2020-08-15 19:05:33,110] - obspy.clients.fdsn.mass_downloader - INFO: Total 

{'BGR': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xb4a503ac>,
 'ETH': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xaa61046c>,
 'GEONET': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xa8cf234c>,
 'GFZ': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xa8cf23ec>,
 'ICGC': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xa8d778ec>,
 'INGV': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xa8c32e8c>,
 'IPGP': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xb4a51f6c>,
 'IRIS': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xa8dcb6ec>,
 'KNMI': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xaa5f6aec>,
 'KOERI': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0xa8d142ac>,
 'LMU': <o

This code loops through each of the downloaded waveforms.  It removes the instrument response using a defined pre-filter and converts to displacement.  The maximum displacement is determined, along with the distance from the event to the station.  The local magnitude is calculated using the approximate emprical formula from Bullen and Bolt (1985).  The average magnitude for each station-channel is reported at the end.

In [31]:
filt = ( 0.5, 1, 40, 50 )
ms = 0.0
n = 0
for file in os.listdir("waveforms"):
    wavfile = "waveforms/" + file
    stafile = "stations/" + file.split(".")[0] + "." + file.split(".")[1] + ".xml"
    print (wavfile, stafile)
    st = read(wavfile)
    tr = st[0]
    inv = read_inventory(stafile)
    tr.remove_response(inventory=inv, pre_filt=filt, output="DISP")
    disp = abs( tr.max() * 1e6 ) # displacement in microns
    stalat = inv.networks[0].stations[0].latitude
    stalon = inv.networks[0].stations[0].longitude
    evelat = cat[0].origins[0].latitude
    evelon = cat[0].origins[0].longitude
    dist = gps2dist_azimuth(stalat, stalon, evelat, evelon )[0] / 1000 # distance in kilometers
    magl = log10(disp) + 2.56 * log10(dist) - 1.67
    print("displacement =",disp,", distance =",dist,", local mag =",magl)
    ms += magl
    n += 1
print ()

waveforms/N4.T50A.00.HH2__20200120T191201Z__20200120T191711Z.mseed stations/N4.T50A.xml
displacement = 4.72844085438 , distance = 97.67877404188674 , local mag = 4.09860647349545
waveforms/N4.T50A.00.HH1__20200120T191201Z__20200120T191711Z.mseed stations/N4.T50A.xml
displacement = 5.28454780806 , distance = 97.67877404188674 , local mag = 4.146896343668285
waveforms/US.TZTN.00.BH1__20200120T191201Z__20200120T191711Z.mseed stations/US.TZTN.xml
displacement = 18.5878703733 , distance = 44.62745650628028 , local mag = 3.822211101793128
waveforms/IM.TKL..BHN__20200120T191201Z__20200120T191711Z.mseed stations/IM.TKL.xml
displacement = 5.06964557298 , distance = 88.69248944279852 , local mag = 4.021567921256405
waveforms/US.TZTN.00.BH2__20200120T191201Z__20200120T191711Z.mseed stations/US.TZTN.xml
displacement = 13.1302136143 , distance = 44.62745650628028 , local mag = 3.671253258271914
waveforms/IM.TKL..BHE__20200120T191201Z__20200120T191711Z.mseed stations/IM.TKL.xml
displacement = 13.607